In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop
from os import listdir

In [2]:
# O is for Organic Foods 
# R is for Recyclable Foods

# Connect to base directory of Project Data
base_dir = 'C:/Users/umara/Desktop/HTN/Covid'

# Create a directory for our Training and Validation Images Data
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test')

# Directory with our training pictures
train_covid_dir = os.path.join(train_dir, 'Covid')
train_normal_dir = os.path.join(train_dir, 'Normal')
train_pneumonia_dir = os.path.join(train_dir, 'Viral Pneumonia')

# Directory with our validation pictures
validation_covid_dir = os.path.join(validation_dir, 'Covid')
validation_normal_dir = os.path.join(validation_dir, 'Normal')
validation_pneumonia_dir = os.path.join(validation_dir, 'Viral Pneumonia')

In [18]:
model = tf.keras.models.Sequential([
    # First Layer
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (150, 150, 3)),
    tf.keras.layers.MaxPool2D(2, 2),
    
    # Second layer
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    
    # Third Layer 
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPool2D(2, 2),

    # Final Layer
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(3, activation = 'softmax')
    
])

In [19]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 18496)            

In [20]:
model.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['accuracy'])

# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen  = ImageDataGenerator( rescale = 1.0/255. )

# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))     
# --------------------
# Flow validation images in batches of 20 using test_datagen generator
# --------------------
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'categorical',
                                                         target_size = (150, 150))

Found 251 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [21]:
class_names = list(train_generator.class_indices.keys())
print(class_names)

['Covid', 'Normal', 'Viral Pneumonia']


In [22]:
history = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=12,
                              epochs=15,
                              validation_steps = 3,
                              verbose=2)

Epoch 1/15
12/12 - 6s - loss: 1.2302 - accuracy: 0.5931 - val_loss: 0.8354 - val_accuracy: 0.8167
Epoch 2/15
12/12 - 5s - loss: 0.5051 - accuracy: 0.8701 - val_loss: 0.4893 - val_accuracy: 0.7333
Epoch 3/15
12/12 - 5s - loss: 0.2110 - accuracy: 0.9091 - val_loss: 0.3181 - val_accuracy: 0.8500
Epoch 4/15
12/12 - 5s - loss: 0.1810 - accuracy: 0.9437 - val_loss: 0.3515 - val_accuracy: 0.8500
Epoch 5/15
12/12 - 6s - loss: 0.1899 - accuracy: 0.9221 - val_loss: 0.3070 - val_accuracy: 0.8500
Epoch 6/15
12/12 - 6s - loss: 0.1224 - accuracy: 0.9524 - val_loss: 0.3183 - val_accuracy: 0.8500
Epoch 7/15
12/12 - 6s - loss: 0.0908 - accuracy: 0.9697 - val_loss: 0.2057 - val_accuracy: 0.8833
Epoch 8/15
12/12 - 6s - loss: 0.0719 - accuracy: 0.9740 - val_loss: 0.5172 - val_accuracy: 0.8500
Epoch 9/15
12/12 - 5s - loss: 0.0349 - accuracy: 0.9827 - val_loss: 0.3009 - val_accuracy: 0.9167
Epoch 10/15
12/12 - 6s - loss: 0.0320 - accuracy: 0.9870 - val_loss: 0.3212 - val_accuracy: 0.9167
Epoch 11/15
12/12 -

In [16]:
import pathlib
from pathlib import Path

In [26]:
model.save('model.h5')

In [27]:

file_name = 'C:/Users/umara/Desktop/HTN/Covid/test/Viral Pneumonia/0101.jpeg'

img = keras.preprocessing.image.load_img(
    file_name, target_size=(150, 150)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to Viral Pneumonia with a 57.61 percent confidence.
